In [1]:
import sys
import os

In [2]:
# parent_dir = "/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024"
parent_dir="/Volumes/work/phd/MoLFormer_N2024"

sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)


In [3]:
import pandas as pd
import numpy as np
import sys
import os
from sklearn.linear_model import Ridge
from utils.prepare_datasets import prepare_dataset
from sklearn.model_selection import cross_validate,train_test_split
from utils.helper_methods import pipeline_regression,custom_linear_regression, metrics_per_descritor,custom_ridge_regression
from utils.util_alignment import grand_average,average_over_subject,post_process_results_df
from constants import *

MoLFormer_N2024


In [4]:
seed = 2024

# Helper methods

In [5]:
def remove_chemistry_representation(chemistry_property_X, representations_y):
    #todo Train-test split how to do it?
    #todo cross-validation to find the best alpha
    reg = Ridge(alpha=1.0)
    reg.fit(chemistry_property_X, representations_y)
    predicted_representation_y = reg.predict(chemistry_property_X)
    return representations_y - predicted_representation_y

In [6]:
# parent_dir = "/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024"
parent_dir="/Volumes/work/phd/MoLFormer_N2024"
sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)
# base_path = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/'
base_path = '../../../../T5 EVO/alignment_olfaction_datasets/'

# GSLF

# Keller

In [7]:
# input_file_keller_pom = base_path+'curated_datasets/embeddings/pom/keller_pom_embeddings_Apr17.csv'
# input_file_keller_dam = base_path+'curated_datasets/alva/keller_molecules_alva_17Apr.csv'
# input_file_keller_dam_mordred = base_path+'curated_datasets/mordred/keller_molecules_mordred_17Apr.csv'
# input_file_keller_molformer = base_path+'curated_datasets/embeddings/molformer/keller_molformer_embeddings_'
# input_file_keller_molformerfinetuned = base_path+'curated_datasets/embeddings/molformerfinetuned/keller_molformerfinetuned_embeddings_',

# 
# dam = pd.read_csv(input_file_keller_dam)
# dam_nonstereo = dam.filter(regex='^nonStereoSMILES|CID')
# molformer = pd.read_csv(input_file_keller_molformer+'13_Apr17.csv')
# molformer = prepare_dataset(molformer, 'embeddings', 'y')



In [8]:
# molformer_filtered

In [9]:
# molformer_filtered_averaged=grand_average(molformer,"keller")
# molformer_filtered= molformer_filtered_averaged[['CID', 'embeddings','y']]
# keller_merged = pd.merge(dam_nonstereo, molformer_filtered, on='CID')

In [10]:
# def prepare_chems_reps_df(input_file_dam, input_file_molformer, i ):
#     dam = pd.read_csv(input_file_dam)
#     dam_nonstereo = dam.filter(regex='^nonStereoSMILES|CID')
#     molformer = pd.read_csv(input_file_molformer+str(i)+'_Apr17.csv')
#     molformer = prepare_dataset(molformer, 'embeddings', 'y')
#     molformer_filtered= molformer[['CID', 'embeddings','y','subject']]
#     keller_merged = pd.merge(dam_nonstereo, molformer_filtered, on='CID')
#     return keller_merged
# 


In [11]:
def save_predictions(df_predictions, ds,title,df_predictions_old):
    if ds.startswith("keller"):
        tasks = keller_tasks
    elif ds.startswith("sagar"):
        tasks = sagar_tasks
    else:
        raise ValueError("Invalid dataset")
    df_predictions = df_predictions.rename(columns=dict(
        zip(df_predictions.columns[1:len(tasks) + 1], [tasks[i] + "_predicted" for i in range(len(tasks))])))
    df_predictions = df_predictions.rename(columns=dict(zip(df_predictions.columns[len(tasks) + 1:len(tasks) * 2 + 1],
                                                            [tasks[i] + "_true" for i in range(len(tasks))])))
    df_predictions_old = df_predictions_old._append(df_predictions, ignore_index=True)
    df_predictions_old.to_csv('dfprediction_'+ds+'_'+title+'.csv', index=False)
    return df_predictions_old

In [12]:
def train_and_eval(data_groupbyCID,feature,times,n_components=None,y_i=None):
    mserrorrs_corssvalidated = []
    correlations_corssvalidated = []
    predicteds = []
    y_tests = []
    runs = []
    CIDs = []
    
    
    X=np.asarray(data_groupbyCID[feature].values.tolist())
    X=X.reshape(-1,1)
    y=np.asarray(data_groupbyCID.embeddings.values.tolist())
    
    # print(X.shape)
    # print(y.shape)
    # print(X)
    # print(y)
    
    
    linreg_fist,X_reduced = pipeline_regression(X,y,X,custom_linear_regression,seed,n_components=None)
    if linreg_fist is None:
        return CIDs,predicteds,y_tests,runs,mserrorrs_corssvalidated, correlations_corssvalidated
    y_predicted_by_chemicals=linreg_fist.predict(X_reduced)
    y_all_left_out = y - y_predicted_by_chemicals
    
    X=y_all_left_out
    y=np.asarray(data_groupbyCID.y.values.tolist())
    
    for i in range(times):
        print("times",i)
        X_train, X_test, y_train, y_test,CID_train, CID_test = train_test_split(X, y,data_groupbyCID.CID, test_size=0.2, random_state=seed+i) 
        linreg,X_test = pipeline_regression(X_train,y_train,X_test,custom_linear_regression,seed,n_components=n_components)
        
        predicted, mseerrors, correlations=metrics_per_descritor(X_test,y_test,linreg)
        mserrorrs_corssvalidated.append(mseerrors)
        correlations_corssvalidated.append(correlations)
        predicteds.extend(predicted)
        y_tests.extend(y_test)
        runs.extend([i]*len(y_test))
        CIDs.extend(CID_test)
        
        
    return CIDs,predicteds,y_tests,runs,mserrorrs_corssvalidated, correlations_corssvalidated

In [13]:
def post_process_tocsv(corrs,tasks):
    
    # print(corrs.columns)
    corrs.columns = tasks+["model","removed_feature", "layer"]    
    return corrs 

In [14]:
def save_data(ds,title,df_cor,df_mse,df):
    if ds=="keller":
        tasks= keller_tasks
    elif ds=="sagar":
        tasks= sagar_tasks
    else:
        raise ValueError("Invalid dataset")
    


    corrs_molfomerfinetuned_df = post_process_tocsv(df_cor,tasks)
    corrs_molfomerfinetuned_df['type'] = 'correlation'
    df = df._append(corrs_molfomerfinetuned_df, ignore_index=True)
      

    mses_molfomerfinetuned_df = post_process_tocsv(df_mse,tasks)
    mses_molfomerfinetuned_df['type'] = 'mse'
    df = df._append(mses_molfomerfinetuned_df, ignore_index=True)
    
    
    df.to_csv('df_'+ds+'_'+title+'.csv', index=False) 
    
    
    return df 


In [18]:
def pipeline(model_name,dam_nonstereo, input_file_molformer,layer,times=30,n_components=None,ds="keller"):
    
    molformer = pd.read_csv(input_file_molformer)
    molformer = prepare_dataset(molformer, 'embeddings', 'y')
    molformer=grand_average(molformer,ds)
    molformer_filtered= molformer[['CID', 'embeddings','y']]
    merged = pd.merge(dam_nonstereo, molformer_filtered, on='CID')
    
    #todo
    features = merged.columns.values.tolist()[2:-2]
    
    
    #create an empty pandas dataframe to store the results
    df_df_all_mse = pd.DataFrame()
    df_df_all_corrs = pd.DataFrame()
    df_df_all_predictions = pd.DataFrame()
    
    try:
        df = pd.read_csv('df_'+ds+'_'+model_name+'.csv')
    except FileNotFoundError:
        df = pd.DataFrame()
        
    try:
        df_prediction_old = pd.read_csv('dfprediction_'+ds+'_'+model_name+'.csv')
    except FileNotFoundError:
        df_prediction_old = pd.DataFrame()
    f_start = 85
    for f, feature in enumerate(features):
        if f < f_start:
            continue
        print("feature: ",feature,f)
    
        CIDs, predicteds, y_tests,runs, mserrorrs_df_corssvalidated, correlations_df_corssvalidated=train_and_eval(merged,feature,times=times,n_components=n_components)
    
        if len(mserrorrs_df_corssvalidated)==0:
            continue
        mserrorrs_corssvalidated_df,statistics_correlations_corssvalidated_df,pvalues_correlations_corssvalidated_df=post_process_results_df(mserrorrs_df_corssvalidated, correlations_df_corssvalidated)
        df_df_mse= pd.DataFrame(mserrorrs_corssvalidated_df)
        df_df_mse['model'] = model_name
        df_df_mse['removed_feature'] = feature
        df_df_mse["layer"]=layer
        # df_df_mse["run"]=runs[0]
       
        df_df_cor= pd.DataFrame(statistics_correlations_corssvalidated_df)
        df_df_cor['model'] = model_name
        df_df_cor['removed_feature'] = feature
        
        df_df_cor["layer"]=layer
        # df_df_cor["run"]=runs[0]
        
        
        
        # I want to make a dataframe with the predicted values, the true values and the run number for each prediction, (192, 22) (192, 22) (192,) should be converted to (196, 22+22+1), 
        df_predictions = pd.DataFrame(np.concatenate([np.asarray(CIDs).reshape(-1,1),np.asarray(predicteds),np.asarray(y_tests),np.asarray(runs).reshape(-1,1)],axis=1))
        df_predictions['model'] = model_name
        df_predictions['removed_feature'] = feature
        df_predictions['layer'] = layer
        #and add a prefix to the columns to indicate the predicted vs true values
        tasks_length = len(sagar_tasks) if ds.startswith('sagar') else len(keller_tasks)
        df_predictions.columns = ['CID']+[str(i)+'_predicted' for i in range(tasks_length)]+[str(i)+'_true' for i in range(tasks_length,int(tasks_length*2))]+['run', 'model', 'removed_feature','layer']
        
        df_df_all_mse = pd.concat([df_df_all_mse,df_df_mse])
        df_df_all_corrs = pd.concat([df_df_all_corrs,df_df_cor])
        df_df_all_predictions = pd.concat([df_df_all_predictions,df_predictions])
        
        
        df = save_data(ds,model_name,df_df_cor,df_df_mse,df)
        df_prediction_old = save_predictions(df_predictions, ds,model_name,df_prediction_old)
    # return df_df_cor,df_df_mse,df_predictions

        
        
    return df_df_all_corrs,df_df_all_mse,df_df_all_predictions

In [16]:
def compute_correlation(times,n_components,input_file,ds="keller"):
    corrs=[]
    mses=[]
    preds = []
    input_file_keller_dam = base_path+'curated_datasets/alva/keller_molecules_alva_17Apr.csv'
    dam = pd.read_csv(input_file_keller_dam)
    dam_nonstereo = dam.filter(regex='^nonStereoSMILES|CID')
    
    for i in [13]:
        input_file_molformer  = input_file+str(i)+'_Apr17.csv'
        # print("input_file_molformer",input_file_molformer)
        df_predictions,df_keller_mse, df_keller_cor = pipeline('molformerorth',dam_nonstereo,input_file_molformer,layer=i,times=times,n_components=n_components,ds=ds)
        
        # df_predictions['layer'] = i
        corrs.append(df_keller_cor)
        mses.append(df_keller_mse)
        preds.append(df_predictions)
        # 

    return corrs,mses,df_predictions

In [ ]:
input_file_keller_molformer = base_path+'curated_datasets/embeddings/molformer/keller_molformer_embeddings_'
d1,d2,d3 = compute_correlation(30,20,input_file_keller_molformer,ds="keller")

feature:  nonStereoSMILES___ZM1 85
Error in fitting
feature:  nonStereoSMILES___ZM1V 86
Error in fitting
feature:  nonStereoSMILES___ZM1Kup 87
Error in fitting
feature:  nonStereoSMILES___ZM1Mad 88
Error in fitting
feature:  nonStereoSMILES___ZM1Per 89
Error in fitting
feature:  nonStereoSMILES___ZM1MulPer 90
Error in fitting
feature:  nonStereoSMILES___ZM2 91
Error in fitting
feature:  nonStereoSMILES___ZM2V 92
Error in fitting
feature:  nonStereoSMILES___ZM2Kup 93
Error in fitting
feature:  nonStereoSMILES___ZM2Mad 94
Error in fitting
feature:  nonStereoSMILES___ZM2Per 95
Error in fitting
feature:  nonStereoSMILES___ZM2MulPer 96
Error in fitting
feature:  nonStereoSMILES___ON0 97
Error in fitting
feature:  nonStereoSMILES___ON0V 98
Error in fitting
feature:  nonStereoSMILES___ON1 99
Error in fitting
feature:  nonStereoSMILES___ON1V 100
Error in fitting
feature:  nonStereoSMILES___Qindex 101
Error in fitting
feature:  nonStereoSMILES___BBI 102
Error in fitting
feature:  nonStereoSMILE

In [ ]:
input_file_sagar_molformer = base_path+'curated_datasets/embeddings/molformer/sagar_molformer_embeddings_'
compute_correlation(30,20,input_file_keller_molformer,ds="sagar")

In [ ]:
\